In [ ]:
# 1. Install & import
!pip install -q geemap rasterio matplotlib cartopy geopandas -q
import ee
import geemap
import matplotlib.pyplot as plt
from matplotlib.patches import Patch
import rasterio
from rasterio.plot import show
import numpy as np
from datetime import datetime
import os

ee.Authenticate()   # ← one-time Google login
ee.Initialize()

In [ ]:
import ee

# Replace with your verified Project ID
PROJECT_ID = 'project-b75af78d-ff46-4d69-ab3'  # e.g., 'earthengine-legacy' or new one

ee.Authenticate()  # If needed
ee.Initialize(project=PROJECT_ID)

print("Success! Earth Engine is ready.")
# Test: print(ee.Image('NASA/NASADEM_HGT/001').getInfo())

In [ ]:
ee.Initialize(project='project-b75af78d-ff46-4d69-ab3')

In [ ]:
import ee

PROJECT_ID = 'project-b75af78d-ff46-4d69-ab3'

# Authenticate if token expired (browser popup)
ee.Authenticate()

# Initialize with your registered project
ee.Initialize(project=PROJECT_ID)

print("Earth Engine initialized successfully!")

# Quick test: load a public global dataset
test = ee.Image('NASA/NASADEM_HGT/001').getInfo()
print("Test image metadata:", test)

In [ ]:
# ================================================
# GODAVARI DELTA FIGURE 1 - 4 PANEL (ORIGINAL)
# Run in Google Colab - produces your own Figure 1
# ================================================

# 1. Install & import
!pip install -q geemap rasterio matplotlib cartopy geopandas -q
import ee
import geemap
import matplotlib.pyplot as plt
from matplotlib.patches import Patch
import rasterio
from rasterio.plot import show
import numpy as np
from datetime import datetime
import os

ee.Authenticate()   # ← one-time Google login
ee.Initialize()

# 2. Define Godavari Delta AOI (precise bounding box from literature)
aoi = ee.Geometry.Rectangle([81.4, 15.9, 83.1, 17.6])

# 3. Panel a: Recent true-color Sentinel-2 + mangrove overlay
s2 = ee.ImageCollection('COPERNICUS/S2_SR_HARMONIZED') \
    .filterBounds(aoi) \
    .filterDate('2024-01-01', '2026-02-19') \
    .filter(ee.Filter.lt('CLOUDY_PIXEL_PERCENTAGE', 20)) \
    .median().clip(aoi)

# Global Mangrove Watch v3 (public in GEE)
gmw = ee.ImageCollection('projects/sat-io/open-datasets/GMW/GMW_V3') \
    .filterDate('2020-01-01', '2020-12-31').mosaic().clip(aoi)

# 4. Panel b: Subsidence (Ohenhen et al. 2026 - download once)
# Download from Zenodo: https://zenodo.org/records/18092985  (file: Godavari_VLM_75m.tif or full delta subset)
# Place the .tif in your Colab /content/ folder or mount Drive
subsidence_path = '/content/godavari_vlm.tif'  # ← change to your filename after upload

# 5. Panel c: Simple NDWI-based shoreline change (1990-2025 vs recent)
# Landsat 5/8 for long-term, Sentinel-2 recent
def add_ndwi(image):
    ndwi = image.normalizedDifference(['B3', 'B8']).rename('NDWI')  # Green-NIR for water
    return image.addBands(ndwi)

ls = ee.ImageCollection('LANDSAT/LC08/C02/T1_L2') \
    .filterBounds(aoi).filterDate('2013-01-01', '2025-12-31') \
    .map(add_ndwi).median().select('NDWI').clip(aoi)

s2_ndwi = s2.normalizedDifference(['B3', 'B8']).rename('NDWI').clip(aoi)

# Difference (simple proxy for shoreline retreat)
change = s2_ndwi.subtract(ls).clip(aoi)

# 6. Create 4-panel figure
fig, axs = plt.subplots(2, 2, figsize=(14, 12), dpi=300)
fig.suptitle('Figure 1 | Coupled subsidence and sediment-starved erosion in the Godavari Delta, India\n(Generated ' + datetime.now().strftime('%Y-%m-%d') + ' via GEE + Zenodo data)', fontsize=14, y=0.98)

# Panel a - Regional context
Map = geemap.Map()
Map.addLayer(s2, {'bands': ['B4', 'B3', 'B2'], 'min': 0, 'max': 3000}, 'True-color 2024-26')
Map.addLayer(gmw, {'palette': ['#006400']}, 'GMW Mangroves 2020', opacity=0.7)
Map.centerObject(aoi, 9)
# For static export we use matplotlib instead of interactive - replace with your screenshot or use geemap.to_image

# Better static version:
ax = axs[0, 0]
# (We export composite to Drive and read back for consistency - code below does it)
print('Exporting Panel a composite to Drive...')

# Export functions (run once)
task_a = ee.batch.Export.image.toDrive(
    image=s2.visualize(**{'bands': ['B4','B3','B2'], 'min':0, 'max':3000}),
    description='Godavari_S2_composite',
    scale=10, region=aoi, maxPixels=1e10)
task_a.start()

# (Continue with other panels similarly; in practice run exports then plot)

# For full working version I recommend splitting into sections.
# Full notebook with all exports + final 4-panel matplotlib assembly is available if you want the expanded version.

print('Script ready. Run the exports, download rasters, then use the matplotlib code below to assemble Figure 1.')

In [ ]:
import ee

# Your registered project ID from earlier (copy exactly as shown in console)
PROJECT_ID = 'project-b75af78d-ff46-4d69-ab3'

# Authenticate (only needed if token expired or first time in session)
ee.Authenticate()  # ← browser popup if required

# Initialize WITH the project argument
ee.Initialize(project=PROJECT_ID)

print("Earth Engine successfully initialized with your registered project!")

In [ ]:
# Simple test to confirm access
test_image = ee.Image('NASA/NASADEM_HGT/001')
info = test_image.getInfo()
print("Test successful! Image info:", info)

In [ ]:
!pip install -q geemap rasterio matplotlib cartopy geopandas

In [ ]:
import ee
import geemap
import matplotlib.pyplot as plt
import rasterio
from rasterio.plot import show
import numpy as np
from datetime import datetime

# === YOUR REGISTERED PROJECT ID ===
PROJECT_ID = 'project-b75af78d-ff46-4d69-ab3'  # copy exactly from your console

# Authenticate (only if token expired or first run in this session)
ee.Authenticate()  # ← this opens browser popup if needed

# Initialize with project (critical!)
ee.Initialize(project=PROJECT_ID)

print("Earth Engine initialized successfully with project:", PROJECT_ID)
print("Ready for geospatial operations!")

In [ ]:
# Test: load a small global image and get its info
test_image = ee.Image('NASA/NASADEM_HGT/001').select('elevation')
info = test_image.getInfo()
print("Test successful! Image info:", info)

# If this prints a dictionary with 'type': 'Image', bands, etc. → everything is working

In [ ]:
# Godavari Delta AOI (bounding box from literature, covers delta front & Coringa)
aoi = ee.Geometry.Rectangle([81.4, 15.9, 83.1, 17.6])

# Recent Sentinel-2 median composite (2024–2025, low cloud)
s2 = ee.ImageCollection('COPERNICUS/S2_SR_HARMONIZED') \
    .filterBounds(aoi) \
    .filterDate('2024-01-01', '2025-12-31') \
    .filter(ee.Filter.lt('CLOUDY_PIXEL_PERCENTAGE', 20)) \
    .median().clip(aoi)

print("Sentinel-2 composite loaded. Bands available:", s2.bandNames().getInfo())

# Global Mangrove Watch (2020 extent as overlay)
gmw = ee.ImageCollection('projects/sat-io/open-datasets/GMW/GMW_V3') \
    .filterDate('2020-01-01', '2020-12-31').mosaic().clip(aoi)

print("Mangrove layer loaded.")

In [ ]:
# Export true-color visualization (for Panel a)
task_a = ee.batch.Export.image.toDrive(
    image=s2.visualize(bands=['B4', 'B3', 'B2'], min=0, max=3000),
    description='Godavari_S2_truecolor_2024-2025',
    folder='Godavari_Figures',  # creates folder in your Drive
    scale=10,
    region=aoi,
    maxPixels=1e10
)
task_a.start()

print("Export started for Panel a. Check Google Drive → Godavari_Figures folder in ~5–15 min.")
print("Task ID:", task_a.id)

In [ ]:
import rasterio
from rasterio.plot import show
import matplotlib.pyplot as plt

try:
    with rasterio.open(full_path) as src:
        print("Successfully opened!")
        print("CRS (Coordinate Reference System):", src.crs)
        print("Width x Height:", src.width, "x", src.height)
        print("Number of bands:", src.count)
        print("Bounds:", src.bounds)
        print("Metadata:", src.meta)

        # Quick visual preview (first band)
        fig, ax = plt.subplots(figsize=(8, 8))
        show(src, ax=ax, cmap='terrain' if 'elevation' in src.descriptions else 'viridis')  # adjust cmap
        ax.set_title("Preview of GeoTIFF")
        plt.show()

except Exception as e:
    print("Error opening file:", str(e))

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

# Adjust path if folder name/location differs
tif_path = '/content/drive/MyDrive/Godavari_Figures/Godavari_S2_truecolor_2024-2025.tif'

import rasterio
from rasterio.plot import show
import matplotlib.pyplot as plt

with rasterio.open(tif_path) as src:
    print("Opened successfully!")
    print("CRS:", src.crs)
    print("Dimensions:", src.width, "×", src.height)
    fig, ax = plt.subplots(figsize=(10, 10))
    show(src, ax=ax)
    ax.set_title("Panel a: True-color Sentinel-2 composite")
    plt.show()

In [ ]:
import rasterio
from rasterio.plot import show
import matplotlib.pyplot as plt
from affine import Affine  # needed for correct scaling

tif_path = '/content/drive/MyDrive/Godavari_Figures/Godavari_S2_truecolor_2024-2025.tif'

with rasterio.open(tif_path) as src:
    print("Metadata:", src.meta)
    print("CRS:", src.crs)
    print("Bounds:", src.bounds)

    # Downsample factor (higher number = smaller image in memory)
    preview_scale = 100  # adjust to 50 or 200 if still too large/slow

    # Correct way to scale the transform for downsampling
    scaled_transform = src.transform * Affine.scale(preview_scale, preview_scale)

    # Read downsampled version
    preview = src.read(
        out_shape=(
            src.count,
            int(src.height / preview_scale),
            int(src.width / preview_scale)
        ),
        resampling=rasterio.enums.Resampling.average  # smooth downsampling
    )

    # Plot the preview
    fig, ax = plt.subplots(figsize=(10, 10))
    show(preview, ax=ax, transform=scaled_transform, title="Downsampled Preview")
    ax.set_title("Downsampled Preview of Panel a (true-color Sentinel-2 composite)")
    plt.show()

In [ ]:
import rasterio
from rasterio.plot import show
import matplotlib.pyplot as plt

# Update this path/filename
subsidence_path = '/content/drive/MyDrive/Godavari_Figures/godavari_vlm.tif'  # ← change to your uploaded file

with rasterio.open(subsidence_path) as src:
    print("Subsidence Metadata:", src.meta)
    print("CRS:", src.crs)
    print("Bounds:", src.bounds)
    print("Units (likely mm/yr):", src.descriptions)  # check if metadata has units

    # Downsampled preview to avoid crash
    preview_scale = 20  # adjust if needed (lower = more detail, more memory)
    scaled_transform = src.transform * Affine.scale(preview_scale, preview_scale)
    preview = src.read(out_shape=(
        src.count,
        int(src.height / preview_scale),
        int(src.width / preview_scale)
    ), resampling=rasterio.enums.Resampling.average)

    fig, ax = plt.subplots(figsize=(10, 10))
    show(preview, ax=ax, transform=scaled_transform, cmap='RdYlBu_r')  # red = subsidence, blue = uplift
    ax.set_title("Downsampled Preview of Panel b (Subsidence Rates, mm/yr)")
    plt.colorbar(ax.images[0], ax=ax, label='Vertical Land Motion (mm/yr)')
    plt.show()

In [ ]:
import rasterio
import numpy as np

subsidence_path = '/content/drive/MyDrive/Godavari_Figures/godavari_vlm.tif'  # ← UPDATE THIS PATH/FILENAME

with rasterio.open(subsidence_path) as src:
    print("Full metadata:", src.meta)
    print("CRS:", src.crs)
    print("Bounds:", src.bounds)
    print("Number of bands:", src.count)
    print("Data type:", src.dtypes[0])
    print("Nodata value:", src.nodata)

    # Read full first (or only) band - but only statistics, not full array
    data = src.read(1)  # band 1
    print("\nData statistics:")
    print("Min value:", np.nanmin(data))
    print("Max value:", np.nanmax(data))
    print("Mean value:", np.nanmean(data))
    print("Number of valid pixels:", np.count_nonzero(~np.isnan(data)))
    print("Percentage valid:", (np.count_nonzero(~np.isnan(data)) / data.size) * 100, "%")
    print("Unique values sample:", np.unique(data[~np.isnan(data)])[:10])  # first 10 unique values

In [ ]:
import rasterio
import numpy as np
import matplotlib.pyplot as plt

subsidence_path = '/content/drive/MyDrive/Godavari_Figures/godavari_vlm.tif'  # your file

with rasterio.open(subsidence_path) as src:
    data = src.read(1, masked=True)
    data = np.ma.masked_invalid(data)

    valid_data = data[~data.mask]
    print("Valid data range:", valid_data.min(), "to", valid_data.max())
    print("Mean:", valid_data.mean())

    fig, ax = plt.subplots(figsize=(10, 8))
    im = ax.imshow(data, extent=[src.bounds.left, src.bounds.right,
                                 src.bounds.bottom, src.bounds.top],
                   cmap='RdYlBu_r', vmin=-5, vmax=1, interpolation='nearest')
    ax.set_title("Panel b Placeholder: Subsidence Rates (1 km res from Zenodo)")
    ax.set_xlabel('Longitude (°E)')
    ax.set_ylabel('Latitude (°N)')
    plt.colorbar(im, ax=ax, label='Vertical Land Motion (mm/yr)')
    plt.tight_layout()
    plt.savefig('Panel_b_placeholder.png', dpi=300, bbox_inches='tight')
    plt.show()

In [ ]:
from google.colab import drive
drive.mount('/content/drive')


In [ ]:
import rasterio
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.patches import Patch
from affine import Affine

# Paths - update if folder name differs
drive_folder = '/content/drive/MyDrive/Godavari_Figures/'
s2_path = drive_folder + 'Godavari_S2_truecolor_2024-2025.tif'
vlm_path = drive_folder + 'godavari_vlm.tif'

fig, axs = plt.subplots(2, 2, figsize=(16, 14), dpi=300)
fig.suptitle('Figure 1 | Coupled Subsidence and Sediment-Starved Erosion in the Godavari Delta\n(Draft with 1 km subsidence placeholder – high-res pending)', fontsize=16, y=0.98)

# Panel a: True-color Sentinel-2 (downsampled to prevent crash)
with rasterio.open(s2_path) as src:
    preview_scale = 50  # adjust higher (e.g. 100) if still crashes
    preview = src.read(out_shape=(src.count, int(src.height/preview_scale), int(src.width/preview_scale)),
                       resampling=rasterio.enums.Resampling.average)
    scaled_transform = src.transform * Affine.scale(preview_scale, preview_scale)
    show(preview, ax=axs[0,0], transform=scaled_transform)
axs[0,0].set_title('(a) Regional satellite context & mangrove extent (Sentinel-2, 2024–2025)')
axs[0,0].set_xlabel('Longitude (°E)')
axs[0,0].set_ylabel('Latitude (°N)')

# Panel b: Subsidence placeholder (full view, small raster)
with rasterio.open(vlm_path) as src:
    data = src.read(1, masked=True)
    data = np.ma.masked_invalid(data)
    im = axs[0,1].imshow(data, extent=[src.bounds.left, src.bounds.right,
                                       src.bounds.bottom, src.bounds.top],
                         cmap='RdYlBu_r', vmin=-5, vmax=1, interpolation='nearest')
axs[0,1].set_title('(b) Land subsidence rates (1 km res placeholder from Zenodo)')
axs[0,1].set_xlabel('Longitude (°E)')
axs[0,1].set_ylabel('Latitude (°N)')
cbar = fig.colorbar(im, ax=axs[0,1], orientation='vertical', fraction=0.046, pad=0.04)
cbar.set_label('Vertical Land Motion (mm/yr)')

# Panel c: Placeholder (NDWI change – we can compute real one next)
axs[1,0].text(0.5, 0.5, 'Panel (c) Shoreline & NDWI Change\n(1990–2025 erosion rates, to be exported from GEE)',
              ha='center', va='center', fontsize=12, color='gray')
axs[1,0].set_title('(c) Multi-decadal shoreline migration')
axs[1,0].axis('off')

# Panel d: Placeholder (vulnerability)
axs[1,1].text(0.5, 0.5, 'Panel (d) Integrated vulnerability hotspots\n(Subsidence + erosion + mangrove loss priority zones)',
              ha='center', va='center', fontsize=12, color='gray')
axs[1,1].set_title('(d) Coupled vulnerability hotspots')
axs[1,1].axis('off')

# Simple legend
legend_elements = [
    Patch(facecolor='red', label='High subsidence (–4 to –5 mm/yr)'),
    Patch(facecolor='blue', label='Stable / minimal subsidence')
]
fig.legend(handles=legend_elements, loc='lower center', ncol=2, bbox_to_anchor=(0.5, -0.02))

plt.tight_layout(rect=[0, 0.03, 1, 0.95])
plt.savefig(drive_folder + 'Draft_Figure1_Godavari_Delta.svg', format='svg', bbox_inches='tight', dpi=300)
print("Draft SVG saved to:", drive_folder + 'Draft_Figure1_Godavari_Delta.svg')
plt.show()

In [ ]:
import pandas as pd

csv_path = '/content/drive/MyDrive/Godavari_Figures/VLM_godavari_Ohenhen data.csv'

df = pd.read_csv(csv_path)
print("Shape:", df.shape)
print("\nColumns:", df.columns.tolist())
print("\nFirst 5 rows:\n", df.head())
print("\nSummary statistics:\n", df.describe())

# Look for key columns (likely something like lat, lon, vlm_mm_yr or rate)
# If column names are unclear, share the output and we'll identify them

In [ ]:
import pandas as pd
import numpy as np
from scipy.interpolate import griddata
import rasterio
from rasterio.transform import from_origin
from rasterio.crs import CRS

# Paths
csv_path = '/content/drive/MyDrive/Godavari_Figures/VLM_godavari_Ohenhen data.csv'
output_raster = '/content/drive/MyDrive/Godavari_Figures/Godavari_VLM_highres.tif'

# Load CSV
df = pd.read_csv(csv_path)
lon = df['longitude'].values
lat = df['latitude'].values
vlm_cm = df['VLM_cm_per_Year'].values

# Convert to mm/yr for consistency with literature
vlm_mm = vlm_cm * 10.0

# Define output grid (~75 m resolution)
res_deg = 0.000675  # ~75 m at equator (adjust to 0.0009 for ~100 m if memory tight)
left, bottom, right, top = 81.4, 16.2, 82.5, 17.1  # slightly expanded to cover all points
nx = int((right - left) / res_deg) + 1
ny = int((top - bottom) / res_deg) + 1

xi = np.linspace(left, right, nx)
yi = np.linspace(bottom, top, ny)
xi, yi = np.meshgrid(xi, yi)

# Interpolate (linear; 'nearest' if gaps are large)
zi = griddata((lon, lat), vlm_mm, (xi, yi), method='linear', fill_value=np.nan)

# Optional: mask outside convex hull (reduces edge artifacts)
from scipy.spatial import ConvexHull
hull = ConvexHull(np.column_stack((lon, lat)))
# (advanced masking code can be added later if needed)

# Save as GeoTIFF (north-up)
transform = from_origin(left, top, res_deg, -res_deg)  # negative y-res for north-up
with rasterio.open(
    output_raster, 'w',
    driver='GTiff',
    height=ny, width=nx,
    count=1, dtype='float32',
    crs=CRS.from_epsg(4326),
    transform=transform,
    nodata=np.nan
) as dst:
    dst.write(zi.astype('float32'), 1)

print(f"High-res VLM raster saved: {output_raster}")
print(f"Grid size: {nx} × {ny} (~75 m resolution)")

In [ ]:
import rasterio
import numpy as np
import matplotlib.pyplot as plt

highres_path = '/content/drive/MyDrive/Godavari_Figures/Godavari_VLM_highres.tif'

with rasterio.open(highres_path) as src:
    data = src.read(1, masked=True)
    valid_data = data[~data.mask]

    print("High-res VLM statistics (mm/yr):")
    print("Min:", np.nanmin(valid_data))
    print("Max:", np.nanmax(valid_data))
    print("Mean:", np.nanmean(valid_data))
    print("Std:", np.nanstd(valid_data))

    fig, ax = plt.subplots(figsize=(12, 10))
    im = ax.imshow(data, extent=[src.bounds.left, src.bounds.right,
                                 src.bounds.bottom, src.bounds.top],
                   cmap='RdYlBu_r', vmin=-50, vmax=10, interpolation='nearest')
    ax.set_title("Panel b: High-resolution Subsidence Rates\n(Ohenhen CSV interpolated ~75 m)")
    ax.set_xlabel('Longitude (°E)')
    ax.set_ylabel('Latitude (°N)')
    cbar = plt.colorbar(im, ax=ax, label='Vertical Land Motion (mm/yr)')
    plt.tight_layout()
    plt.savefig('/content/drive/MyDrive/Godavari_Figures/Panel_b_highres_preview.png', dpi=300, bbox_inches='tight')
    plt.show()

In [ ]:
import rasterio
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.patches import Patch
from affine import Affine

drive_folder = '/content/drive/MyDrive/Godavari_Figures/'
s2_path = drive_folder + 'Godavari_S2_truecolor_2024-2025.tif'
vlm_highres_path = drive_folder + 'Godavari_VLM_highres.tif'

fig, axs = plt.subplots(2, 2, figsize=(16, 14), dpi=300)
fig.suptitle('Figure 1 | Subsidence Outpaces Sea-Level Rise and Amplifies Sediment-Starved Erosion in the Godavari Delta\n(High-res VLM from Ohenhen et al. 2026 – Panels c/d pending)', fontsize=16, y=0.98)

# Panel a: S2 true-color (downsampled to avoid memory issues)
with rasterio.open(s2_path) as src:
    preview_scale = 50  # increase to 100 if still crashes
    preview = src.read(out_shape=(src.count, int(src.height/preview_scale), int(src.width/preview_scale)),
                       resampling=rasterio.enums.Resampling.average)
    scaled_transform = src.transform * Affine.scale(preview_scale, preview_scale)
    show(preview, ax=axs[0,0], transform=scaled_transform)
axs[0,0].set_title('(a) Regional satellite context & mangrove extent (Sentinel-2, 2024–2025)')
axs[0,0].set_xlabel('Longitude (°E)')
axs[0,0].set_ylabel('Latitude (°N)')

# Panel b: High-res subsidence
with rasterio.open(vlm_highres_path) as src:
    data = src.read(1, masked=True)
    im = axs[0,1].imshow(data, extent=[src.bounds.left, src.bounds.right,
                                       src.bounds.bottom, src.bounds.top],
                         cmap='RdYlBu_r', vmin=-50, vmax=10, interpolation='nearest')
axs[0,1].set_title('(b) High-resolution land subsidence rates (~75 m, Ohenhen data)')
axs[0,1].set_xlabel('Longitude (°E)')
axs[0,1].set_ylabel('Latitude (°N)')
cbar = fig.colorbar(im, ax=axs[0,1], orientation='vertical', fraction=0.046, pad=0.04)
cbar.set_label('Vertical Land Motion (mm/yr)')

# Panel c & d: placeholders (replace with real data next)
axs[1,0].text(0.5, 0.5, 'Panel (c) Shoreline Change & NDWI Difference\n(1990–2025 erosion rates, to be exported from GEE)',
              ha='center', va='center', fontsize=12, color='gray')
axs[1,0].set_title('(c) Multi-decadal shoreline migration')
axs[1,0].axis('off')

axs[1,1].text(0.5, 0.5, 'Panel (d) Integrated vulnerability hotspots\n(Subsidence + erosion + mangrove degradation priority zones)',
              ha='center', va='center', fontsize=12, color='gray')
axs[1,1].set_title('(d) Coupled vulnerability hotspots')
axs[1,1].axis('off')

# Legend
legend_elements = [
    Patch(facecolor='red', label='Extreme subsidence (–20 to –47 mm/yr)'),
    Patch(facecolor='orange', label='Moderate subsidence (–5 to –20 mm/yr)'),
    Patch(facecolor='blue', label='Stable / minor change')
]
fig.legend(handles=legend_elements, loc='lower center', ncol=3, bbox_to_anchor=(0.5, -0.02), fontsize=10)

plt.tight_layout(rect=[0, 0.03, 1, 0.95])
plt.savefig(drive_folder + 'Figure1_HighRes_VLM_Draft.svg', format='svg', bbox_inches='tight', dpi=300)
print("Updated SVG with high-res Panel b saved:", drive_folder + 'Figure1_HighRes_VLM_Draft.svg')
plt.show()

In [ ]:
import ee
from datetime import datetime

# Re-initialize (if session restarted)
PROJECT_ID = 'project-b75af78d-ff46-4d69-ab3'
ee.Initialize(project=PROJECT_ID)

# AOI (same as before)
aoi = ee.Geometry.Rectangle([81.4, 15.9, 83.1, 17.6])

# Function to add NDWI (Green - NIR normalized difference)
def add_ndwi(image):
    ndwi = image.normalizedDifference(['SR_B3', 'SR_B5']).rename('NDWI')  # Landsat 8/9 bands
    return image.addBands(ndwi)

# Older period: Landsat 5/7 (1990–2010 median, pre-major dam effects)
ls_old = ee.ImageCollection('LANDSAT/LT05/C02/T1_L2') \
    .merge(ee.ImageCollection('LANDSAT/LE07/C02/T1_L2')) \
    .filterBounds(aoi) \
    .filterDate('1990-01-01', '2010-12-31') \
    .filter(ee.Filter.lt('CLOUD_COVER', 20)) \
    .map(add_ndwi) \
    .select('NDWI') \
    .median().clip(aoi)

# Recent period: Landsat 8/9 + Sentinel-2 (2015–2025 median)
ls_recent = ee.ImageCollection('LANDSAT/LC08/C02/T1_L2') \
    .merge(ee.ImageCollection('LANDSAT/LC09/C02/T1_L2')) \
    .filterBounds(aoi) \
    .filterDate('2015-01-01', '2025-12-31') \
    .filter(ee.Filter.lt('CLOUD_COVER', 20)) \
    .map(add_ndwi) \
    .select('NDWI') \
    .median().clip(aoi)

# Change: older NDWI - recent NDWI (positive = gain/water increase, negative = loss/land gain or erosion)
ndwi_change = ls_old.subtract(ls_recent).rename('NDWI_Change')

# Visualization for preview (export will be full res)
vis_params = {'min': -0.5, 'max': 0.5, 'palette': ['red', 'white', 'blue']}  # red=loss/erosion, blue=gain

# Export to Drive
task_c = ee.batch.Export.image.toDrive(
    image=ndwi_change,
    description='Godavari_NDWI_Change_1990-2025',
    folder='Godavari_Figures',
    scale=30,  # Landsat resolution
    region=aoi,
    maxPixels=1e10
)
task_c.start()

print("Export started for Panel c NDWI change. Task ID:", task_c.id)
print("Check Tasks tab at https://code.earthengine.google.com/tasks")
print("File will appear in Drive → Godavari_Figures → Godavari_NDWI_Change_1990-2025.tif (wait 10–60 min)")

In [ ]:
import rasterio
import numpy as np
import matplotlib.pyplot as plt

ndwi_path = '/content/drive/MyDrive/Godavari_Figures/Godavari_NDWI_Change_1990-2025.tif'

with rasterio.open(ndwi_path) as src:
    data = src.read(1, masked=True)
    valid_data = data[~data.mask]

    print("NDWI Change statistics (older - recent):")
    print("Min (strong loss/erosion):", np.nanmin(valid_data))
    print("Max (strong gain/accretion):", np.nanmax(valid_data))
    print("Mean:", np.nanmean(valid_data))
    print("Std:", np.nanstd(valid_data))

    fig, ax = plt.subplots(figsize=(12, 10))
    im = ax.imshow(data, extent=[src.bounds.left, src.bounds.right,
                                 src.bounds.bottom, src.bounds.top],
                   cmap='RdBu_r', vmin=-0.5, vmax=0.5, interpolation='nearest')
    ax.set_title("Panel c: NDWI Change (1990–2025)\nRed = loss/erosion (shoreline retreat), Blue = gain")
    ax.set_xlabel('Longitude (°E)')
    ax.set_ylabel('Latitude (°N)')
    cbar = plt.colorbar(im, ax=ax, label='NDWI Change (dimensionless)')
    plt.tight_layout()
    plt.savefig('/content/drive/MyDrive/Godavari_Figures/Panel_c_NDWI_preview.png', dpi=300, bbox_inches='tight')
    plt.show()

In [ ]:
import rasterio
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.patches import Patch
from affine import Affine

drive_folder = '/content/drive/MyDrive/Godavari_Figures/'
s2_path     = drive_folder + 'Godavari_S2_truecolor_2024-2025.tif'
vlm_path    = drive_folder + 'Godavari_VLM_highres.tif'
ndwi_path   = drive_folder + 'Godavari_NDWI_Change_1990-2025.tif'

fig, axs = plt.subplots(2, 2, figsize=(16, 14), dpi=300)
fig.suptitle('Figure 1 | Subsidence Outpaces Sea-Level Rise and Amplifies Sediment-Starved Erosion in the Godavari Delta', fontsize=16, y=0.98)

# Panel a: S2 true-color (downsampled)
with rasterio.open(s2_path) as src:
    preview_scale = 50
    preview = src.read(out_shape=(src.count, int(src.height/preview_scale), int(src.width/preview_scale)),
                       resampling=rasterio.enums.Resampling.average)
    scaled_transform = src.transform * Affine.scale(preview_scale, preview_scale)
    show(preview, ax=axs[0,0], transform=scaled_transform)
axs[0,0].set_title('(a) Regional satellite context (Sentinel-2, 2024–2025)')
axs[0,0].set_xlabel('Longitude (°E)')
axs[0,0].set_ylabel('Latitude (°N)')

# Panel b: High-res subsidence
with rasterio.open(vlm_path) as src:
    data = src.read(1, masked=True)
    im_b = axs[0,1].imshow(data, extent=[src.bounds.left, src.bounds.right,
                                         src.bounds.bottom, src.bounds.top],
                           cmap='RdYlBu_r', vmin=-50, vmax=10, interpolation='nearest')
axs[0,1].set_title('(b) High-resolution subsidence rates (~75 m)')
axs[0,1].set_xlabel('Longitude (°E)')
axs[0,1].set_ylabel('Latitude (°N)')
cbar_b = fig.colorbar(im_b, ax=axs[0,1], orientation='vertical', fraction=0.046, pad=0.04)
cbar_b.set_label('Vertical Land Motion (mm/yr)')

# Panel c: NDWI change
with rasterio.open(ndwi_path) as src:
    data = src.read(1, masked=True)
    im_c = axs[1,0].imshow(data, extent=[src.bounds.left, src.bounds.right,
                                         src.bounds.bottom, src.bounds.top],
                           cmap='RdBu_r', vmin=-0.5, vmax=0.5, interpolation='nearest')
axs[1,0].set_title('(c) NDWI change (1990–2025)')
axs[1,0].set_xlabel('Longitude (°E)')
axs[1,0].set_ylabel('Latitude (°N)')
cbar_c = fig.colorbar(im_c, ax=axs[1,0], orientation='vertical', fraction=0.046, pad=0.04)
cbar_c.set_label('NDWI Change (older - recent)')

# Panel d: Placeholder (simple overlay or text)
axs[1,1].text(0.5, 0.5, 'Panel (d) Integrated vulnerability hotspots\n(Subsidence + NDWI change + erosion priority zones)',
              ha='center', va='center', fontsize=12, color='gray')
axs[1,1].set_title('(d) Coupled vulnerability hotspots')
axs[1,1].axis('off')

# Legend
legend_elements = [
    Patch(facecolor='red',   label='Strong subsidence / erosion'),
    Patch(facecolor='blue',  label='Stable / gain')
]
fig.legend(handles=legend_elements, loc='lower center', ncol=2, bbox_to_anchor=(0.5, -0.02), fontsize=10)

plt.tight_layout(rect=[0, 0.03, 1, 0.95])
plt.savefig(drive_folder + 'Figure1_Complete_Draft_with_PanelC.svg', format='svg', bbox_inches='tight', dpi=300)
print("Final draft SVG saved:", drive_folder + 'Figure1_Complete_Draft_with_PanelC.svg')
plt.show()

In [ ]:
# Assuming high-res VLM and NDWI TIFFs are aligned (same extent/res)
vlm_path = '/content/drive/MyDrive/Godavari_Figures/Godavari_VLM_highres.tif'
ndwi_path = '/content/drive/MyDrive/Godavari_Figures/Godavari_NDWI_Change_1990-2025.tif'

with rasterio.open(vlm_path) as vlm_src, rasterio.open(ndwi_path) as ndwi_src:
    vlm_data = vlm_src.read(1, masked=True)
    ndwi_data = ndwi_src.read(1, masked=True)

    # Normalize (simple min-max)
    vlm_norm = (vlm_data - np.nanmin(vlm_data)) / (np.nanmax(vlm_data) - np.nanmin(vlm_data))
    ndwi_norm = (ndwi_data - np.nanmin(ndwi_data)) / (np.nanmax(ndwi_data) - np.nanmin(ndwi_data))

    # Vulnerability index: high subsidence + high erosion (negative NDWI change)
    # Invert NDWI so negative change = high vulnerability
    vuln_index = vlm_norm + (1 - ndwi_norm)  # higher = more vulnerable
    vuln_index = np.clip(vuln_index, 0, 2)  # scale 0–2

    # Save as new TIFF
    out_meta = vlm_src.meta.copy()
    out_meta.update(dtype=rasterio.float32, count=1)
    with rasterio.open('/content/drive/MyDrive/Godavari_Figures/Godavari_Vulnerability_Hotspots.tif', 'w', **out_meta) as dst:
        dst.write(vuln_index.astype('float32'), 1)

print("Panel d vulnerability index saved.")

In [ ]:
import rasterio
from rasterio.warp import calculate_default_transform, reproject, Resampling
import numpy as np

vlm_path = '/content/drive/MyDrive/Godavari_Figures/Godavari_VLM_highres.tif'
ndwi_path = '/content/drive/MyDrive/Godavari_Figures/Godavari_NDWI_Change_1990-2025.tif'
output_vuln_path = '/content/drive/MyDrive/Godavari_Figures/Godavari_Vulnerability_Hotspots.tif'

# 1. Open VLM (target grid)
with rasterio.open(vlm_path) as vlm_src:
    vlm_data = vlm_src.read(1, masked=True)
    vlm_meta = vlm_src.meta.copy()
    vlm_transform = vlm_src.transform
    vlm_shape = vlm_data.shape

# 2. Open NDWI and reproject to match VLM grid
with rasterio.open(ndwi_path) as ndwi_src:
    # Prepare destination array
    ndwi_reproj = np.empty(vlm_shape, dtype=np.float32)

    # Reproject NDWI into VLM grid
    reproject(
        source=rasterio.band(ndwi_src, 1),
        destination=ndwi_reproj,
        src_transform=ndwi_src.transform,
        src_crs=ndwi_src.crs,
        dst_transform=vlm_transform,
        dst_crs=vlm_src.crs,
        resampling=Resampling.average,  # smooth downsampling
    )

    ndwi_data = np.ma.masked_invalid(ndwi_reproj)

# 3. Normalize both (min-max, handle masks)
def normalize(arr):
    valid = arr[~arr.mask]
    if valid.size == 0:
        return np.zeros_like(arr)
    return (arr - valid.min()) / (valid.max() - valid.min())

vlm_norm = normalize(vlm_data)
ndwi_norm = normalize(ndwi_data)

# 4. Vulnerability index: high subsidence (negative VLM) + high erosion (negative NDWI change)
# Invert NDWI so negative change → high vulnerability
vuln_index = (-vlm_norm) + (-ndwi_norm)   # both negative → positive vulnerability
vuln_index = np.clip(vuln_index, 0, 2)     # scale 0–2

# 5. Save as GeoTIFF (copy VLM metadata)
vuln_meta = vlm_meta.copy()
vuln_meta.update(dtype=rasterio.float32, nodata=np.nan)

with rasterio.open(output_vuln_path, 'w', **vuln_meta) as dst:
    dst.write(vuln_index.astype('float32'), 1)

print(f"Panel d vulnerability hotspots saved: {output_vuln_path}")

In [ ]:
import rasterio
import numpy as np
import matplotlib.pyplot as plt

vuln_path = '/content/drive/MyDrive/Godavari_Figures/Godavari_Vulnerability_Hotspots.tif'

with rasterio.open(vuln_path) as src:
    data = src.read(1, masked=True)
    valid = data[~data.mask]

    print("Vulnerability Index statistics (0–2, higher = more vulnerable):")
    print("Min:", np.nanmin(valid))
    print("Max:", np.nanmax(valid))
    print("Mean:", np.nanmean(valid))
    print("Std:", np.nanstd(valid))

    fig, ax = plt.subplots(figsize=(10, 8))
    im = ax.imshow(data, extent=[src.bounds.left, src.bounds.right,
                                 src.bounds.bottom, src.bounds.top],
                   cmap='OrRd', vmin=0, vmax=2, interpolation='nearest')
    ax.set_title("Panel (d) Vulnerability Hotspots\n(Subsidence + NDWI Change)")
    ax.set_xlabel('Longitude (°E)')
    ax.set_ylabel('Latitude (°N)')
    plt.colorbar(im, ax=ax, label='Relative Vulnerability (0–2)')
    plt.tight_layout()
    plt.savefig('/content/drive/MyDrive/Godavari_Figures/Panel_d_vuln_preview.png', dpi=300, bbox_inches='tight')
    plt.show()

In [ ]:
import rasterio
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.patches import Patch
from affine import Affine

drive_folder = '/content/drive/MyDrive/Godavari_Figures/'
s2_path   = drive_folder + 'Godavari_S2_truecolor_2024-2025.tif'
vlm_path  = drive_folder + 'Godavari_VLM_highres.tif'
ndwi_path = drive_folder + 'Godavari_NDWI_Change_1990-2025.tif'
vuln_path = drive_folder + 'Godavari_Vulnerability_Hotspots.tif'

fig, axs = plt.subplots(2, 2, figsize=(16, 14), dpi=300)
fig.suptitle('Figure 1 | Subsidence Outpaces Sea-Level Rise and Amplifies Sediment-Starved Erosion in the Godavari Delta', fontsize=16, y=0.98)

# Panel a: S2 true-color (downsampled)
with rasterio.open(s2_path) as src:
    preview_scale = 50
    preview = src.read(out_shape=(src.count, int(src.height/preview_scale), int(src.width/preview_scale)),
                       resampling=rasterio.enums.Resampling.average)
    scaled_transform = src.transform * Affine.scale(preview_scale, preview_scale)
    show(preview, ax=axs[0,0], transform=scaled_transform)
axs[0,0].set_title('(a) Regional satellite context (Sentinel-2, 2024–2025)')
axs[0,0].set_xlabel('Longitude (°E)')
axs[0,0].set_ylabel('Latitude (°N)')

# Panel b: High-res subsidence
with rasterio.open(vlm_path) as src:
    data = src.read(1, masked=True)
    im_b = axs[0,1].imshow(data, extent=[src.bounds.left, src.bounds.right,
                                         src.bounds.bottom, src.bounds.top],
                           cmap='RdYlBu_r', vmin=-50, vmax=10, interpolation='nearest')
axs[0,1].set_title('(b) High-resolution subsidence rates (~75 m)')
axs[0,1].set_xlabel('Longitude (°E)')
axs[0,1].set_ylabel('Latitude (°N)')
cbar_b = fig.colorbar(im_b, ax=axs[0,1], orientation='vertical', fraction=0.046, pad=0.04)
cbar_b.set_label('Vertical Land Motion (mm/yr)')

# Panel c: NDWI change
with rasterio.open(ndwi_path) as src:
    data = src.read(1, masked=True)
    im_c = axs[1,0].imshow(data, extent=[src.bounds.left, src.bounds.right,
                                         src.bounds.bottom, src.bounds.top],
                           cmap='RdBu_r', vmin=-0.5, vmax=0.5, interpolation='nearest')
axs[1,0].set_title('(c) NDWI change (1990–2025)')
axs[1,0].set_xlabel('Longitude (°E)')
axs[1,0].set_ylabel('Latitude (°N)')
cbar_c = fig.colorbar(im_c, ax=axs[1,0], orientation='vertical', fraction=0.046, pad=0.04)
cbar_c.set_label('NDWI Change (older - recent)')

# Panel d: Vulnerability hotspots
with rasterio.open(vuln_path) as src:
    data = src.read(1, masked=True)
    im_d = axs[1,1].imshow(data, extent=[src.bounds.left, src.bounds.right,
                                         src.bounds.bottom, src.bounds.top],
                           cmap='OrRd', vmin=0, vmax=2, interpolation='nearest')
axs[1,1].set_title('(d) Integrated vulnerability hotspots')
axs[1,1].set_xlabel('Longitude (°E)')
axs[1,1].set_ylabel('Latitude (°N)')
cbar_d = fig.colorbar(im_d, ax=axs[1,1], orientation='vertical', fraction=0.046, pad=0.04)
cbar_d.set_label('Relative Vulnerability (0–2)')

# Legend
legend_elements = [
    Patch(facecolor='red',   label='High subsidence / erosion / vulnerability'),
    Patch(facecolor='blue',  label='Stable / gain / low risk')
]
fig.legend(handles=legend_elements, loc='lower center', ncol=2, bbox_to_anchor=(0.5, -0.02), fontsize=10)

plt.tight_layout(rect=[0, 0.03, 1, 0.95])
plt.savefig(drive_folder + 'Figure1_Final_Draft_All_Panels.svg', format='svg', bbox_inches='tight', dpi=300)
print("Final SVG with all panels saved:", drive_folder + 'Figure1_Final_Draft_All_Panels.svg')
plt.show()

In [ ]:
import rasterio
import numpy as np
import matplotlib.pyplot as plt

vlm_path = '/content/drive/MyDrive/Godavari_Figures/Godavari_VLM_highres.tif'
ndwi_path = '/content/drive/MyDrive/Godavari_Figures/Godavari_NDWI_Change_1990-2025.tif'
output_vuln_path = '/content/drive/MyDrive/Godavari_Figures/Godavari_Vulnerability_Hotspots_Improved.tif'

# Load and reproject NDWI to VLM grid (as before)
with rasterio.open(vlm_path) as vlm_src:
    vlm_data = vlm_src.read(1, masked=True)
    vlm_meta = vlm_src.meta.copy()
    vlm_transform = vlm_src.transform

with rasterio.open(ndwi_path) as ndwi_src:
    ndwi_reproj = np.empty(vlm_data.shape, dtype=np.float32)
    reproject(
        source=rasterio.band(ndwi_src, 1),
        destination=ndwi_reproj,
        src_transform=ndwi_src.transform,
        src_crs=ndwi_src.crs,
        dst_transform=vlm_transform,
        dst_crs=vlm_src.crs,
        resampling=Resampling.average,
    )
    ndwi_data = np.ma.masked_invalid(ndwi_reproj)

# Improved index: threshold-based (more sensitive to extremes)
# High vulnerability if subsidence < -5 mm/yr AND NDWI change < -0.1
vuln_binary = np.zeros_like(vlm_data, dtype=np.float32)
vuln_binary[(vlm_data < -5) & (ndwi_data < -0.1)] = 2.0   # high
vuln_binary[(vlm_data < -5) | (ndwi_data < -0.1)] = 1.0   # medium
vuln_binary = np.ma.masked_where(vlm_data.mask | ndwi_data.mask, vuln_binary)

# Save
with rasterio.open(output_vuln_path, 'w', **vlm_meta) as dst:
    dst.write(vuln_binary, 1)

print("Improved threshold-based vulnerability saved.")

# Preview
with rasterio.open(output_vuln_path) as src:
    data = src.read(1, masked=True)
    fig, ax = plt.subplots(figsize=(10, 8))
    im = ax.imshow(data, extent=[src.bounds.left, src.bounds.right,
                                 src.bounds.bottom, src.bounds.top],
                   cmap='OrRd', vmin=0, vmax=2, interpolation='nearest')
    ax.set_title("Improved Panel d: Vulnerability Hotspots (Threshold-based)")
    plt.colorbar(im, ax=ax, label='Vulnerability Level (0=low, 1=medium, 2=high)')
    plt.show()

In [ ]:
import rasterio
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.patches import Patch
from affine import Affine

drive_folder = '/content/drive/MyDrive/Godavari_Figures/'
s2_path   = drive_folder + 'Godavari_S2_truecolor_2024-2025.tif'
vlm_path  = drive_folder + 'Godavari_VLM_highres.tif'
ndwi_path = drive_folder + 'Godavari_NDWI_Change_1990-2025.tif'
vuln_path = drive_folder + 'Godavari_Vulnerability_Hotspots_Improved.tif'  # ← new improved version

fig, axs = plt.subplots(2, 2, figsize=(16, 14), dpi=300)
fig.suptitle('Figure 1 | Subsidence Outpaces Sea-Level Rise and Amplifies Sediment-Starved Erosion in the Godavari Delta', fontsize=16, y=0.98)

# Panel a: S2 true-color (downsampled)
with rasterio.open(s2_path) as src:
    preview_scale = 50
    preview = src.read(out_shape=(src.count, int(src.height/preview_scale), int(src.width/preview_scale)),
                       resampling=rasterio.enums.Resampling.average)
    scaled_transform = src.transform * Affine.scale(preview_scale, preview_scale)
    show(preview, ax=axs[0,0], transform=scaled_transform)
axs[0,0].set_title('(a) Regional satellite context (Sentinel-2, 2024–2025)')
axs[0,0].set_xlabel('Longitude (°E)')
axs[0,0].set_ylabel('Latitude (°N)')

# Panel b: High-res subsidence
with rasterio.open(vlm_path) as src:
    data = src.read(1, masked=True)
    im_b = axs[0,1].imshow(data, extent=[src.bounds.left, src.bounds.right,
                                         src.bounds.bottom, src.bounds.top],
                           cmap='RdYlBu_r', vmin=-50, vmax=10, interpolation='nearest')
axs[0,1].set_title('(b) High-resolution subsidence rates (~75 m)')
axs[0,1].set_xlabel('Longitude (°E)')
axs[0,1].set_ylabel('Latitude (°N)')
cbar_b = fig.colorbar(im_b, ax=axs[0,1], orientation='vertical', fraction=0.046, pad=0.04)
cbar_b.set_label('Vertical Land Motion (mm/yr)')

# Panel c: NDWI change
with rasterio.open(ndwi_path) as src:
    data = src.read(1, masked=True)
    im_c = axs[1,0].imshow(data, extent=[src.bounds.left, src.bounds.right,
                                         src.bounds.bottom, src.bounds.top],
                           cmap='RdBu_r', vmin=-0.5, vmax=0.5, interpolation='nearest')
axs[1,0].set_title('(c) NDWI change (1990–2025)')
axs[1,0].set_xlabel('Longitude (°E)')
axs[1,0].set_ylabel('Latitude (°N)')
cbar_c = fig.colorbar(im_c, ax=axs[1,0], orientation='vertical', fraction=0.046, pad=0.04)
cbar_c.set_label('NDWI Change (older - recent)')

# Panel d: Improved vulnerability hotspots
with rasterio.open(vuln_path) as src:
    data = src.read(1, masked=True)
    im_d = axs[1,1].imshow(data, extent=[src.bounds.left, src.bounds.right,
                                         src.bounds.bottom, src.bounds.top],
                           cmap='OrRd', vmin=0, vmax=2, interpolation='nearest')
axs[1,1].set_title('(d) Integrated vulnerability hotspots')
axs[1,1].set_xlabel('Longitude (°E)')
axs[1,1].set_ylabel('Latitude (°N)')
cbar_d = fig.colorbar(im_d, ax=axs[1,1], orientation='vertical', fraction=0.046, pad=0.04)
cbar_d.set_label('Vulnerability Level (0=low, 1=medium, 2=high)')

# Legend
legend_elements = [
    Patch(facecolor='darkred',  label='High vulnerability (2)'),
    Patch(facecolor='orange',   label='Medium vulnerability (1)'),
    Patch(facecolor='lightyellow', label='Low vulnerability (0)')
]
fig.legend(handles=legend_elements, loc='lower center', ncol=3, bbox_to_anchor=(0.5, -0.02), fontsize=10)

plt.tight_layout(rect=[0, 0.03, 1, 0.95])
plt.savefig(drive_folder + 'Figure1_Final_All_Panels_Improved.svg', format='svg', bbox_inches='tight', dpi=300)
print("Final SVG with improved Panel d saved:", drive_folder + 'Figure1_Final_All_Panels_Improved.svg')
plt.show()